<a href="https://colab.research.google.com/github/Scc8/cuda/blob/main/cudatest1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%cd /usr/local/
!rm -rf cuda
!ln -s /usr/local/cuda-12.2 /usr/local/cuda
%cd /content

/usr/local
/content


添加cuda

In [3]:
!stat cuda

stat: cannot statx 'cuda': No such file or directory


In [4]:
!pip install nvcc4jupyter

In [5]:
%load_ext nvcc4jupyter

Source files will be saved in "/tmp/tmpey466i5z".


In [11]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [13]:
#%%writefile add.cu
%%cuda
#include <stdio.h>

__global__ void hello(){
    printf("Hello from block: %u, thread: %u\n", blockIdx.x, threadIdx.x);
}

int main(){
    hello<<<3, 3>>>();

    cudaDeviceSynchronize();

}

Hello from block: 2, thread: 0
Hello from block: 2, thread: 1
Hello from block: 2, thread: 2
Hello from block: 0, thread: 0
Hello from block: 0, thread: 1
Hello from block: 0, thread: 2
Hello from block: 1, thread: 0
Hello from block: 1, thread: 1
Hello from block: 1, thread: 2



In [10]:
%%writefile add.cu
// add.cu
#include <iostream>
#include <math.h>
// Kernel function to add the elements of two arrays
// __global__ 变量声明符，作用是将add函数变成可以在GPU上运行的函数
// __global__ 函数被称为kernel，
// 在 GPU 上运行的代码通常称为设备代码（device code），而在 CPU 上运行的代码是主机代码（host code）。
__global__
void add(int n, float *x, float *y)
{
  for (int i = 0; i < n; i++)
    y[i] = x[i] + y[i];
}

int main(void)
{
  int N = 1<<25;
  float *x, *y;

  // Allocate Unified Memory – accessible from CPU or GPU
  // 内存分配，在GPU或者CPU上统一分配内存
  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));

  // initialize x and y arrays on the host
  for (int i = 0; i < N; i++) {
    x[i] = 1.0f;
    y[i] = 2.0f;
  }

  // Run kernel on 1M elements on the GPU
  // execution configuration, 执行配置
  add<<<1, 1>>>(N, x, y);

  // Wait for GPU to finish before accessing on host
  // CPU需要等待cuda上的代码运行完毕，才能对数据进行读取
  cudaDeviceSynchronize();

  // Check for errors (all values should be 3.0f)
  float maxError = 0.0f;
  for (int i = 0; i < N; i++)
    maxError = fmax(maxError, fabs(y[i]-3.0f));
  std::cout << "Max error: " << maxError << std::endl;

  // Free memory
  cudaFree(x);
  cudaFree(y);
  std::cout << "123123123131231"<< std::endl;
  return 0;
}

Overwriting add.cu


In [12]:

!nvcc add.cu -o add_cuda
!./add_cuda

Max error: 0
123123123131231
